In [ ]:
from pfs.drp.stella import DetectorMap
import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
import lsst.afw.detection as afwDetection
from  pfs.utils.dummyCableB import DummyCableBDatabase

# Define drp repo

In [ ]:
rerun = "sm1-march2020"
repo = "drp"
drpPath = "/home/fmadec/pfs/subaruData"

SM = "SM1"
site = "Subaru"


# Select a visit and a cam

In [ ]:
cam = "r1"

visit = 1376

In [ ]:
arm = cam[0]

In [ ]:
butler = dafPersist.Butler(f"{drpPath}/{repo}/rerun/{rerun}/detrend/")

In [ ]:
calExp = butler.get("calexp", visit=visit, arm=arm)
calexfilePath = butler.getUri("calexp", visit=visit, arm=arm)
print(calexfilePath)

# Retrieve some informations on this visit

## Lamp used

In [ ]:
rbutler = dafPersist.Butler(f"{drpPath}/{repo}/")
rbutler.getKeys('raw')
lamps = rbutler.queryMetadata('raw', ['lamps'], visit=visit, arm=arm) 
print(f"Lamp used: {lamps}")

## PFS Config Id (so fibers used)

In [ ]:
pfsConfig = butler.get("pfsConfig", visit=visit)
pfsConfig.pfsDesignId
dcb = DummyCableBDatabase()
fiberSetup = dcb.interpret(pfsConfig.pfsDesignId)
print(f"DCB fibers config: {fiberSetup}")
fiberIds = dcb.getFiberIds(*fiberSetup)
print(f"Fiber used: {fiberIds}")

# Display image in DS9

## display image

In [ ]:
display = afwDisplay.Display(1)

In [ ]:
# display image
display.setMaskTransparency(70)
display.mtv(calExp, title=f"{visit}")
display.scale("linear", "zscale")

## display variance

In [ ]:
display = afwDisplay.Display(2)

In [ ]:
# display variance
display.setMaskTransparency(70)
display.mtv(calExp.variance, title=f"Variance {visit}")
display.scale("linear", "zscale")

## Change mask 

### Interpreting displayed mask colors


In [ ]:
mask = calExp.getMask()
for maskName, maskBit in mask.getMaskPlaneDict().items():
    print('{}: {}'.format(maskName, display.getMaskPlaneColor(maskName)))

### disable all mask

In [ ]:
mask = calExp.getMask()
for maskName, maskBit in mask.getMaskPlaneDict().items():
    display.setMaskPlaneColor(maskName, "IGNORE")
    print('{}: {}'.format(maskName, display.getMaskPlaneColor(maskName)))

### specify some maskPlane

In [ ]:
maskPlane = "CR"
calExp.getMaskedImage().getMask().addMaskPlane(maskPlane)
display.setMaskPlaneColor(maskPlane, "magenta")
maskPlane = "SAT"
calExp.getMaskedImage().getMask().addMaskPlane(maskPlane)
display.setMaskPlaneColor(maskPlane, "green")

### have to redo display to apply mask change

In [ ]:
display = afwDisplay.Display(1)
# display image
display.setMaskTransparency(70)
display.mtv(calExp, title=f"{visit}")
display.scale("linear", "zscale")